# Resource usage of the StellarGraph class

> This notebooks records the time and memory (both peak and long-term) required to construct a StellarGraph object for several datasets.

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/internal-developers/graph-resource-usage.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/internal-developers/graph-resource-usage.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>

This notebook is aimed at helping contributors to the StellarGraph library itself understand how their changes affect the resource usage of the `StellarGraph` object.

Various measures of resource usage for several "real world" graphs of various sizes are recorded:

- time for construction
- memory usage of the final `StellarGraph` object
- peak memory usage during `StellarGraph` construction (both absolute, and additional compared to the raw input data)

These are recorded both with explicit nodes (and node features if they exist), and implicit/inferred nodes.

The memory usage is recorded end-to-end. That is, the recording starts from data on disk and continues until the `StellarGraph` object has been constructed and other data has been cleaned up. This is important for accurately recording the total memory usage, as NumPy arrays can often share data with existing arrays in memory and so retroactive or partial (starting from data in memory) analysis can miss significant amounts of data. The parsing code in `stellargraph.datasets` doesn't allow determining the memory usage of the intermediate nodes and edges input to the `StellarGraph` constructor, and so cannot be used here.

In [1]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.1.0b

In [2]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.1.0b")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.1.0b, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [3]:
import stellargraph as sg
import pandas as pd
import numpy as np

import gc
import json
import os
import timeit
import tempfile
import tracemalloc

## Optional reddit data

The original GraphSAGE paper evaluated on a reddit dataset, available at <http://snap.stanford.edu/graphsage/#datasets>. This dataset is large (1.3GB compressed) and so there is not automatic download support for it. The following `reddit_path` variable controls whether and how the reddit dataset is included:

- to ignore the dataset: set the variable to `None`
- to include the dataset: download the dataset zip, decompress it, and set the variable to the decompressed directory

In [4]:
reddit_path = os.path.expanduser("~/data/reddit")

## Datasets

### Cora

In [5]:
cora = sg.datasets.Cora()
cora.download()

cora_cites_path = os.path.join(cora.data_directory, "cora.cites")
cora_content_path = os.path.join(cora.data_directory, "cora.content")
cora_dtypes = {0: int, **{i: np.float32 for i in range(1, 1433 + 1)}}


def cora_parts(include_nodes):
    if include_nodes:
        nodes = pd.read_csv(
            cora_content_path,
            header=None,
            sep="\t",
            index_col=0,
            usecols=range(0, 1433 + 1),
            dtype=cora_dtypes,
            na_filter=False,
        )
    else:
        nodes = None
    edges = pd.read_csv(
        cora_cites_path,
        header=None,
        sep="\t",
        names=["source", "target"],
        dtype=int,
        na_filter=False,
    )
    return nodes, edges, {}

### BlogCatalog3

In [6]:
blogcatalog3 = sg.datasets.BlogCatalog3()
blogcatalog3.download()

blogcatalog3_edges = os.path.join(blogcatalog3.data_directory, "edges.csv")
blogcatalog3_group_edges = os.path.join(blogcatalog3.data_directory, "group-edges.csv")
blogcatalog3_groups = os.path.join(blogcatalog3.data_directory, "groups.csv")
blogcatalog3_nodes = os.path.join(blogcatalog3.data_directory, "nodes.csv")


def blogcatalog3_parts(include_nodes):
    if include_nodes:
        raw_nodes = pd.read_csv(blogcatalog3_nodes, header=None)[0]
        raw_groups = pd.read_csv(blogcatalog3_groups, header=None)[0]
        nodes = {
            "user": pd.DataFrame(index=raw_nodes),
            "group": pd.DataFrame(index=-raw_groups),
        }
    else:
        nodes = None

    edges = pd.read_csv(blogcatalog3_edges, header=None, names=["source", "target"])

    group_edges = pd.read_csv(
        blogcatalog3_group_edges, header=None, names=["source", "target"]
    )
    group_edges["target"] *= -1
    start = len(edges)
    group_edges.index = range(start, start + len(group_edges))

    edges = {"friend": edges, "belongs": group_edges}
    return nodes, edges, {}

### FB15k

In [7]:
fb15k = sg.datasets.FB15k()
fb15k.download()
fb15k_files = [
    os.path.join(fb15k.data_directory, f"freebase_mtr100_mte100-{x}.txt")
    for x in ["train", "test", "valid"]
]


def fb15k_parts(include_nodes, usecols=None):
    loaded = [
        pd.read_csv(
            name,
            header=None,
            names=["source", "label", "target"],
            sep="\t",
            dtype=str,
            na_filter=False,
            usecols=usecols,
        )
        for name in fb15k_files
    ]
    edges = pd.concat(loaded, ignore_index=True)

    if include_nodes:
        # infer the set of nodes manually, in a memory-minimal way
        raw_nodes = set(edges.source)
        raw_nodes.update(edges.target)
        nodes = pd.DataFrame(index=raw_nodes)
    else:
        nodes = None

    return nodes, edges, {"edge_type_column": "label"}


def fb15k_no_edge_types_parts(include_nodes):
    nodes, edges, _ = fb15k_parts(include_nodes, usecols=["source", "target"])
    return nodes, edges, {}

### Reddit

As discussed above, the reddit dataset is large and optional. It is also slow to parse, as the graph structure is a huge JSON file. Thus, we prepare the dataset by converting that JSON file into a NumPy edge list array, of shape `(num_edges, 2)`. This is significantly faster to load from disk.

In [8]:
%%time

# if requested, prepare the reddit dataset by saving the slow-to-read JSON to a temporary .npy file
if reddit_path is not None:
    reddit_graph_path = os.path.join(reddit_path, "reddit-G.json")
    reddit_feats_path = os.path.join(reddit_path, "reddit-feats.npy")

    with open(reddit_graph_path) as f:
        reddit_g = json.load(f)
    reddit_numpy_edges = np.array([[x["source"], x["target"]] for x in reddit_g["links"]])
    
    reddit_edges_file = tempfile.NamedTemporaryFile(suffix=".npy")
    np.save(reddit_edges_file, reddit_numpy_edges)

CPU times: user 17.2 s, sys: 1.72 s, total: 18.9 s
Wall time: 18.9 s


In [9]:
def reddit_parts(include_nodes):
    if include_nodes:
        raw_nodes = np.load(reddit_feats_path)
        nodes = pd.DataFrame(raw_nodes)
    else:
        nodes = None

    raw_edges = np.load(reddit_edges_file.name)
    edges = pd.DataFrame(raw_edges, columns=["source", "target"])
    return nodes, edges, {}

### Collected

In [10]:
datasets = {
    "Cora": cora_parts,
    "BlogCatalog3": blogcatalog3_parts,
    "FB15k (no edge types)": fb15k_no_edge_types_parts,
    "FB15k": fb15k_parts,
}
if reddit_path is not None:
    datasets["reddit"] = reddit_parts

## Measurement

In [11]:
def mem_snapshot_diff(after, before):
    """Total memory difference between two tracemalloc.snapshot objects"""
    return sum(elem.size_diff for elem in after.compare_to(before, "lineno"))

In [12]:
# names of columns computed by the measurement code
def measurement_columns(title):
    names = [
        "time",
        "mem (SG)",
        "peak mem (SG)",
        "peak mem (SG above data)",
        "mem (data)",
        "peak mem (data)",
    ]
    return [(title, x) for x in names]


columns = pd.MultiIndex.from_tuples(
    [
        ("graph", "nodes"),
        ("graph", "node feat size"),
        ("graph", "edges"),
        *measurement_columns("explicit nodes"),
        *measurement_columns("inferred nodes (no features)"),
    ]
)
columns

MultiIndex([(                       'graph',                    'nodes'),
            (                       'graph',           'node feat size'),
            (                       'graph',                    'edges'),
            (              'explicit nodes',                     'time'),
            (              'explicit nodes',                 'mem (SG)'),
            (              'explicit nodes',            'peak mem (SG)'),
            (              'explicit nodes', 'peak mem (SG above data)'),
            (              'explicit nodes',               'mem (data)'),
            (              'explicit nodes',          'peak mem (data)'),
            ('inferred nodes (no features)',                     'time'),
            ('inferred nodes (no features)',                 'mem (SG)'),
            ('inferred nodes (no features)',            'peak mem (SG)'),
            ('inferred nodes (no features)', 'peak mem (SG above data)'),
            ('inferred nodes (no featu

In [13]:
def measure_time(f, include_nodes):
    nodes, edges, args = f(include_nodes)
    start = timeit.default_timer()
    sg.StellarGraph(nodes, edges, **args)
    end = timeit.default_timer()
    return end - start

In [14]:
def measure_memory(f, include_nodes):
    """
    Measure exactly what it takes to load the data.
    
    - the size of the original edge data (as a baseline)
    - the size of the final graph
    - the peak memory use of both
    
    This uses a similar technique to the 'allocation_benchmark' fixture in tests/test_utils/alloc.py.
    """
    gc.collect()

    tracemalloc.start()
    snapshot_start = tracemalloc.take_snapshot()

    nodes, edges, args = f(include_nodes)

    gc.collect()
    _, data_memory_peak = tracemalloc.get_traced_memory()
    snapshot_data = tracemalloc.take_snapshot()

    if include_nodes:
        assert nodes is not None, f
        sg_g = sg.StellarGraph(nodes, edges, **args)
    else:
        assert nodes is None, f
        sg_g = sg.StellarGraph(edges=edges, **args)

    # clean up the input data and anything else leftover, so that the snapshot
    # includes only the long-lasting data: the StellarGraph.
    del edges
    del nodes
    del args
    gc.collect()

    _, graph_memory_peak = tracemalloc.get_traced_memory()
    snapshot_end = tracemalloc.take_snapshot()
    tracemalloc.stop()

    data_memory = mem_snapshot_diff(snapshot_data, snapshot_start)
    graph_memory = mem_snapshot_diff(snapshot_end, snapshot_start)

    return (
        sg_g,
        graph_memory,
        graph_memory_peak,
        graph_memory_peak - data_memory,
        data_memory,
        data_memory_peak,
    )

In [15]:
def measure(f):
    time_nodes = measure_time(f, include_nodes=True)
    time_no_nodes = measure_time(f, include_nodes=False)

    sg_g, *mem_nodes = measure_memory(f, include_nodes=True)
    _, *mem_no_nodes = measure_memory(f, include_nodes=False)

    feat_sizes = sg_g.node_feature_sizes()
    try:
        feat_sizes = feat_sizes[sg_g.unique_node_type()]
    except ValueError:
        pass

    return [
        sg_g.number_of_nodes(),
        feat_sizes,
        sg_g.number_of_edges(),
        time_nodes,
        *mem_nodes,
        time_no_nodes,
        *mem_no_nodes,
    ]

In [16]:
%%time
recorded = [measure(f) for f in datasets.values()]

CPU times: user 24.2 s, sys: 5.33 s, total: 29.5 s
Wall time: 29.6 s


In [17]:
raw = pd.DataFrame(recorded, columns=columns, index=datasets.keys())
raw

graph                                     \
                        nodes           node feat size     edges   
Cora                     2708                     1433      5429   
BlogCatalog3            10351  {'user': 0, 'group': 0}    348459   
FB15k (no edge types)   14951                        0    592213   
FB15k                   14951                        0    592213   
reddit                 232965                      602  11606919   

                      explicit nodes                           \
                                time   mem (SG) peak mem (SG)   
Cora                        0.030567   15616268      46772250   
BlogCatalog3                0.045832    6070473      18996022   
FB15k (no edge types)       0.152511    6398752      37222062   
FB15k                       0.776939   12081749      57260717   
reddit                      4.826181  712113351    3551638965   

                                                                            \
                      peak mem (SG above data)  mem (data) peak mem (data)   
Cora                                  31088362    15683888        31993944   
BlogCatalog3                          13326461     5669561        10806213   
FB15k (no edge types)                 21542696    15679366        25831073   
FB15k                                 36358643    20902074        35791845   
reddit                              2243953025  1307685940      1307694919   

                      inferred nodes (no features)                           \
                                              time   mem (SG) peak mem (SG)   
Cora                                      0.006995      94284        405221   
BlogCatalog3                              0.041768    6069969      18996934   
FB15k (no edge types)                     0.244645    6407984      37232686   
FB15k                                     0.867180   12072501      57252173   
reddit                                    1.078231  153909433     624260822   

                                                                           
                      peak mem (SG above data) mem (data) peak mem (data)  
Cora                                    314961      90260          197784  
BlogCatalog3                          13414822    5582112        10711888  
FB15k (no edge types)                 21678832   15553854        25050234  
FB15k                                 36475611   20776562        35011006  
reddit                               438546030  185714792       185723195

## Pretty results

This shows the results in a prettier way, such as memory in MiB instead of bytes.

In [18]:
mem_columns = raw.columns[["mem" in x[1] for x in raw.columns]]

memory_mib = raw.copy()
memory_mib[mem_columns] /= 2 ** 20
memory_mib

graph                                     \
                        nodes           node feat size     edges   
Cora                     2708                     1433      5429   
BlogCatalog3            10351  {'user': 0, 'group': 0}    348459   
FB15k (no edge types)   14951                        0    592213   
FB15k                   14951                        0    592213   
reddit                 232965                      602  11606919   

                      explicit nodes                            \
                                time    mem (SG) peak mem (SG)   
Cora                        0.030567   14.892834     44.605494   
BlogCatalog3                0.045832    5.789254     18.116018   
FB15k (no edge types)       0.152511    6.102325     35.497725   
FB15k                       0.776939   11.522054     54.608075   
reddit                      4.826181  679.124213   3387.106862   

                                                                             \
                      peak mem (SG above data)   mem (data) peak mem (data)   
Cora                                 29.648172    14.957321       30.511803   
BlogCatalog3                         12.709104     5.406915       10.305608   
FB15k (no edge types)                20.544716    14.953009       24.634431   
FB15k                                34.674304    19.933771       34.133763   
reddit                             2140.000367  1247.106495     1247.115058   

                      inferred nodes (no features)                            \
                                              time    mem (SG) peak mem (SG)   
Cora                                      0.006995    0.089916      0.386449   
BlogCatalog3                              0.041768    5.788774     18.116888   
FB15k (no edge types)                     0.244645    6.111130     35.507856   
FB15k                                     0.867180   11.513234     54.599927   
reddit                                    1.078231  146.779473    595.341513   

                                                                            
                      peak mem (SG above data)  mem (data) peak mem (data)  
Cora                                  0.300370    0.086079        0.188622  
BlogCatalog3                         12.793371    5.323517       10.215652  
FB15k (no edge types)                20.674545   14.833311       23.889765  
FB15k                                34.785853   19.814074       33.389097  
reddit                              418.230085  177.111427      177.119441

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/internal-developers/graph-resource-usage.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/internal-developers/graph-resource-usage.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>